In [1]:
%load_ext autoreload
%autoreload 2

import time
import pickle
import numpy as np
from scipy import optimize

import matplotlib.pyplot as plt   
plt.style.use('seaborn-whitegrid')
prop_cycle = plt.rcParams['axes.prop_cycle']
colors = prop_cycle.by_key()['color']

from BabyMAKROModel import BabyMAKROModelClass

# Setup

In [2]:
model = BabyMAKROModelClass(name='baseline') # create an instance of the model

In [3]:
par = model.par
ss = model.ss
sol = model.sol

# Find steady state

In [4]:
model.find_ss(do_print=True)

Labor supply, search and matching:
ss.S/par.N_work = 0.15, ss.L/par.N_work = 0.96, ss.U/par.N_work = 0.04
ss.delta_L = 0.12, ss.m_s = 0.75, ss.m_v = 0.75, ss.v = 7.58
Capital agency FOC:
ss.r_K = 0.14
Labor agency FOC:
ss.r_ell = 1.01, (ss.L-ss.ell)/par.N_work*100 = 0.76
Production firm:
ss.Gamma = 0.57, ss.Y = 80.09, ss.K = 175.64
Capital accumulation:
ss.iota = 17.56, ss.I = 17.56
Government:
ss.B = 0.00, ss.G = 24.03, ss.tau = 0.58
Households:
ss.Aq/par.N = 0.04, ss.C = 26.28, ss.A = 55.00
Re-packing firms:
Market clearing:
ss.C_Y/ss.C = 0.70, ss.G_Y/ss.G = 0.90, ss.I_Y/ss.I = 0.65, ss.X_Y/ss.X = 0.60
ss.X/ss.Y = 0.60, ss.M/ss.Y = 0.44
Bargaining:
par.phi = 0.643


# Speed and error tests

In [5]:
model.set_exo_ss()
model.set_unknowns_ss()

**Python evaluation:**

In [6]:
%time model.evaluate_blocks(py=True)

Wall time: 3.71 s


**Check errors:**

In [7]:
errors = model.get_errors(do_print=True)

A_R_ini_error       : abs. max = 5.34e-15
Aq_diff             : abs. max = 5.33e-14
bargaining_cond     : abs. max = 0.00e+00
FOC_capital_agency  : abs. max = 4.44e-16
FOC_K_ell           : abs. max = 1.78e-15
mkt_clearing        : abs. max = 1.42e-14
PC                  : abs. max = 1.11e-16


In [8]:
if np.allclose(errors,0.0):
    print('errors are zero')
else:
    raise ValueError('errors are NOT zero')

errors are zero


**First numba evaluation:**

In [9]:
%time model.evaluate_blocks()

Wall time: 1min 8s


**Time:**

In [10]:
%timeit model.evaluate_blocks()
%timeit model.get_errors()

20.4 ms ± 2.23 ms per loop (mean ± std. dev. of 7 runs, 100 loops each)
55.5 µs ± 3.02 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [11]:
%timeit model.evaluate_blocks(py=True)

347 ms ± 34.2 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
